In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3


engine = sqlite3.connect('example.db')
def rsq(query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, engine)

In [23]:
rsq("""SELECT UNContinentRegion,
CountryName,
(SELECT AVG(Magnitude) -- Add average magnitude
FROM Earthquakes e
-- Add country code reference
WHERE n.Code2 = e.Country) AS AverageMagnitude
FROM Nations n
ORDER BY UNContinentRegion DESC,
AverageMagnitude DESC;""")

,UNContinentRegion,CountryName,AverageMagnitude
0,Oceania,Solomon Islands,6.934211
1,Oceania,Vanuatu,6.930952
2,Oceania,Papua New Guinea,6.894545
3,Oceania,New Zealand,6.890000
4,Oceania,Fiji,6.850000
...,...,...,...
228,Africa,Togo,NaN
229,Africa,Tunisia,NaN
230,Africa,Uganda,NaN
231,Africa,Western Sahara,NaN


In [24]:
rsq("""SELECT WorldBankRegion,
CountryName
FROM Nations
WHERE Code2 IN -- Add the operator to compare queries
(SELECT CountryCode -- Country code column
FROM Cities);""")

,WorldBankRegion,CountryName
0,South Asia,Afghanistan
1,Europe and Central Asia,Albania
2,Middle East and North Africa,Algeria
3,East Asia and Pacific,American Samoa
4,Europe and Central Asia,Andorra
...,...,...
221,East Asia and Pacific,Vietnam
222,East Asia and Pacific,Wallis and Futuna Islands
223,Middle East and North Africa,Yemen
224,Sub-Sahara Africa,Zambia


In [25]:
rsq("""SELECT WorldBankRegion,
CountryName,
Code2,
Capital, -- Country capital column
Pop2017
FROM Nations AS n
WHERE NOT EXISTS -- Add the operator to compare queries
(SELECT 1
FROM Cities AS c
WHERE n.Code2 = c.CountryCode); -- Columns being compared""")

,WorldBankRegion,CountryName,Code2,Capital,Pop2017
0,Europe and Central Asia,Guernsey,GG,Saint Peter Port,165314
1,Sub-Sahara Africa,Namibia,None,Windhoek,2533794
2,Latin America and Carribean,Netherlands Antilles,AN,Willemstad,25398
3,Middle East and North Africa,Palestinian Territory,PS,Ramallah,4920724
4,Europe and Central Asia,Svalbard and Jan Mayen Islands,SJ,Longyearbyen,2572
5,East Asia and Pacific,Tokelau,TK,Atafu,1300
6,Middle East and North Africa,Western Sahara,EH,Laayoune,552628


In [12]:
rsq("""SELECT WorldBankRegion,
CountryName
FROM Nations
WHERE Code2 NOT IN -- Add the operator to compare queries
(SELECT CountryCode -- Country code column
FROM Cities);""")

,WorldBankRegion,CountryName


In [13]:
rsq("""SELECT WorldBankRegion,
CountryName,
Capital
FROM Nations
WHERE Capital NOT IN
(SELECT NearestPop
FROM Earthquakes
WHERE NearestPop IS NOT NULL); -- filter condition""")

,WorldBankRegion,CountryName,Capital
0,South Asia,Afghanistan,Kabul
1,Europe and Central Asia,Albania,Tirana
2,Middle East and North Africa,Algeria,Algiers
3,East Asia and Pacific,American Samoa,Pago Pago
4,Europe and Central Asia,Andorra,Andorra la Vella
...,...,...,...
224,East Asia and Pacific,Wallis and Futuna Islands,Mata Utu
225,Middle East and North Africa,Western Sahara,Laayoune
226,Middle East and North Africa,Yemen,Sanaa
227,Sub-Sahara Africa,Zambia,Lusaka


In [27]:
rsq("""SELECT WorldBankRegion,
CountryName,
Capital -- Capital city name column
FROM Nations
WHERE Capital  IN
(SELECT NearestPop -- City name column
FROM Earthquakes);""")

,WorldBankRegion,CountryName,Capital
0,Europe and Central Asia,Azerbaijan,Baku
1,Latin America and Carribean,Cayman Islands,George Town
2,Latin America and Carribean,Chile,Santiago
3,East Asia and Pacific,Vanuatu,Port-Vila


In [17]:
rsq("""-- Initial query
SELECT TeamName,
TeamCode,
City
FROM Teams AS t -- Add table
WHERE EXISTS -- Operator to compare queries
(SELECT 1
FROM Earthquakes AS e -- Add table
WHERE t.City = e.NearestPop);""")

,TeamName,TeamCode,City
0,San Antonio Spurs,SAS,San Antonio


In [28]:
rsq("""-- Second query
SELECT t.TeamName,
t.TeamCode,
t.City,
e.Date,
e.Place, -- Place description
e.Country -- Country code
FROM Teams AS t
INNER JOIN Earthquakes AS e -- Operator to compare tables
ON t.City = e.NearestPop""")

,TeamName,TeamCode,City,Date,place,Country
0,San Antonio Spurs,SAS,San Antonio,2019-08-01,"San Antonio, Chile",US


In [ ]:
# SELECT * 
# FROM Orders
# order by OrderID
# offset 300 rows

In [20]:
rsq("""-- First attempt
SELECT c.CustomerID,
c.CompanyName,
c.ContactName,
c.ContactTitle,
c.Phone
FROM Customers c
LEFT OUTER JOIN Orders o -- Joining operator
ON c.CustomerID = o.CustomerID -- Joining columns
WHERE c.Country = 'France';""")

,CustomerID,CompanyName,ContactName,ContactTitle,Phone
0,BLONP,Blondesddsl pere et fils,Frederique Citeaux,Marketing Manager,88.60.15.31
1,BLONP,Blondesddsl pere et fils,Frederique Citeaux,Marketing Manager,88.60.15.31
2,BLONP,Blondesddsl pere et fils,Frederique Citeaux,Marketing Manager,88.60.15.31
3,BLONP,Blondesddsl pere et fils,Frederique Citeaux,Marketing Manager,88.60.15.31
4,BLONP,Blondesddsl pere et fils,Frederique Citeaux,Marketing Manager,88.60.15.31
5,BLONP,Blondesddsl pere et fils,Frederique Citeaux,Marketing Manager,88.60.15.31
6,BLONP,Blondesddsl pere et fils,Frederique Citeaux,Marketing Manager,88.60.15.31
7,BLONP,Blondesddsl pere et fils,Frederique Citeaux,Marketing Manager,88.60.15.31
8,BLONP,Blondesddsl pere et fils,Frederique Citeaux,Marketing Manager,88.60.15.31
9,BONAP,Bon app,Laurence Lebihan,Owner,91.24.45.40


In [21]:
rsq("""-- Second attempt
SELECT c.CustomerID,
c.CompanyName,
c.ContactName,
c.ContactTitle,
c.Phone
FROM Customers c
LEFT OUTER JOIN Orders o
ON c.CustomerID = o.CustomerID
WHERE c.Country = 'France'
AND o.CustomerID IS NULL; -- Filter condition
""")

,CustomerID,CompanyName,ContactName,ContactTitle,Phone
0,FRANR,France restauration,Carine Schmitt,Marketing Manager,40.32.21.21
1,LACOR,La corne d abondance,Daniel Tonini,Sales Representative,30.59.84.10
2,PARIS,Paris specialites,Marie Bertrand,Owner,(1) 42.34.22.66
3,SPECD,Specialites du monde,Dominique Perrier,Marketing Manager,(1) 47.55.60.10


In [22]:
rsq("""-- Second attempt
SELECT c.CustomerID,
c.CompanyName,
c.ContactName,
c.ContactTitle,
c.Phone
FROM Customers c
LEFT OUTER JOIN Orders o
ON c.CustomerID = o.CustomerID
WHERE c.Country = 'France'
AND o.CustomerID IS NULL; -- Filter condition

Test your knowle""")

DatabaseError: Execution failed on sql '-- Second attempt
SELECT c.CustomerID,
c.CompanyName,
c.ContactName,
c.ContactTitle,
c.Phone
FROM Customers c
LEFT OUTER JOIN Orders o
ON c.CustomerID = o.CustomerID
WHERE c.Country = 'France'
AND o.CustomerID IS NULL; -- Filter condition

Test your knowle': You can only execute one statement at a time.